<a href="https://colab.research.google.com/github/danmartin25/Hockey_Model/blob/main/Hockey_Model_Mark_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notes**
1. On-ice EV xG+/-

2. On-ice EV G+/-

3. On-ice PP G+/- above average

4. On-ice SH G+/- above average

5. GSAx

6. Individual points above average (depending on position and role)

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Import Even-Strength On-Ice Totals Data
skater_EV_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/EV%20On-Ice%20Totals.csv')
#print(skater_EV_totals_raw)
#skater_EV_totals_raw.head()

In [3]:
#Restrict Totals Data to GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
skater_EV_totals = skater_EV_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF%','xGF%','GF','GA','xGF','xGA']]
#skater_EV_totals

In [4]:
#Add columns for G+/-,xG+/-
skater_EV_totals['G+/-'] = skater_EV_totals['GF'] - skater_EV_totals['GA']
skater_EV_totals['xG+/-'] = skater_EV_totals['xGF'] - skater_EV_totals['xGA']
#skater_EV_totals

In [5]:
#Import PP On-Ice Rates Data
skater_PP_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/PP%20On-Ice%20Totals.csv')
#print(skater_PP_totals_raw)
#skater_PP_totals_raw.head()

In [6]:
#Reduce PP On-Ice Totals Data
skater_PP_totals = skater_PP_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF%','xGF%','GF','GA','xGF','xGA']]
#skater_PP_totals

In [7]:
#Add columns for G+/-,xG+/-,G/s,xG/s
skater_PP_totals['G+/-'] = skater_PP_totals['GF'] - skater_PP_totals['GA']
skater_PP_totals['xG+/-'] = skater_PP_totals['xGF'] - skater_PP_totals['xGA']
#skater_PP_totals

In [8]:
#Add column for mean G+/- and xG+/-
G_mean = skater_PP_totals['G+/-'].mean()
xG_mean = skater_PP_totals['xG+/-'].mean()
#G_mean
#xG_mean

In [9]:
#Add column for stats above average for G+/- and xG+/-
skater_PP_totals['GAA'] = skater_PP_totals['G+/-'] - G_mean
skater_PP_totals['xGAA'] = skater_PP_totals['xG+/-'] - xG_mean
#skater_PP_totals

In [10]:
#Drop columns
skater_PP_totals = skater_PP_totals.drop(columns = ['xGF%','xGF','xGA','xG+/-','xGAA'])
#skater_PP_totals

In [11]:
#Import SH On-Ice Totals Data
skater_SH_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/SH%20On-Ice%20Totals.csv')
#print(skater_SH_totals_raw)
#skater_SH_totals_raw.head()

In [12]:
#Reduce SH On-Ice Totals Data
skater_SH_totals = skater_SH_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF%','xGF%','GF','GA','xGF','xGA']]
#skater_SH_totals

In [13]:
#Add columns for G+/-,xG+/-
skater_SH_totals['G+/-'] = skater_SH_totals['GF'] - skater_SH_totals['GA']
skater_SH_totals['xG+/-'] = skater_SH_totals['xGF'] - skater_SH_totals['xGA']
#skater_SH_totals

In [14]:
#Add column for mean G+/- and xG+/-
G_mean = skater_SH_totals['G+/-'].mean()
xG_mean = skater_SH_totals['xG+/-'].mean()
#G_mean
#xG_mean

In [15]:
#Add column for stats above average for G+/- and xG+/-
skater_SH_totals['GAA'] = skater_SH_totals['G+/-'] - G_mean
skater_SH_totals['xGAA'] = skater_SH_totals['xG+/-'] - xG_mean
#skater_SH_totals

In [16]:
skater_SH_totals = skater_SH_totals.drop(columns = ['xGF%','xGF','xGA','xG+/-','xGAA'])
#skater_SH_totals

In [17]:
#Import Player Box Stats
skater_box_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Stats%20Box.csv')
#print(skater_box_totals_raw)
#skater_box_totals_raw.head()

In [18]:
#Reduce Data Stats
skater_box_totals = skater_box_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G','A1','A2','Points']]
#skater_box_totals.head()

In [19]:
#Look at positions to get correct F/D average points
#print(skater_box_totals['Position'].unique())
#print(skater_box_totals.count())

forwards_table = skater_box_totals.loc[(skater_box_totals.Position == "L") | (skater_box_totals.Position == "C") | (skater_box_totals.Position == "C/L") | (skater_box_totals.Position == "R") | (skater_box_totals.Position == "L/R") | (skater_box_totals.Position == "C/R")]
#forwards_table.head(10)
#print(forwards_table.count())

defensemen_table = skater_box_totals.loc[(skater_box_totals.Position == "D") | (skater_box_totals.Position == "D/L")]
#defensemen_table.head(10)
#print(defensemen_table.count())

In [20]:
#Get Mean of Points
Points_mean = skater_box_totals['Points'].mean()
#print(Points_mean)

Forwards_mean = forwards_table['Points'].mean()
#print(Forwards_mean)

Defensemen_mean = defensemen_table['Points'].mean()
#print(Defensemen_mean)

In [21]:
#Get PAA
#skater_box_totals['PAA'] = skater_box_totals['Points'] - Points_mean
#skater_box_totals

#Get PAA for Fwds and Dmen and combine

forwards_table['PAA'] = forwards_table['Points'] - Forwards_mean
forwards_table.head(10)

defensemen_table['PAA'] = defensemen_table['Points'] - Defensemen_mean
defensemen_table.head(10)


#Add dataframes back together and sort again

concat_frames = [forwards_table, defensemen_table]
skater_box_totals = pd.concat(concat_frames)
skater_box_totals = skater_box_totals.sort_index()
#skater_box_totals.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [25]:
#Here's where problem is occuring when we combine the tables we're ending up with for example ekblad stats being repeated numerous times
#combined_player_df = pd.merge(skater_EV_totals, skater_PP_totals, how="left", on=["Player", "Player"])
#combined_player_df = skater_EV_totals.merge(skater_PP_totals, on = 'Player', how = 'left')
#combined_player_df.head(20)

combined_player_df = pd.concat([skater_EV_totals,skater_PP_totals], axis='columns')
combined_player_df

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,...,Season,Team,Position,GP,TOI,GF%,GF,GA,G+/-,GAA
0,A.J. Greer,21-22,N.J,L,9,70.45,50.50,58.74,3.01,2.95,...,19-20,FLA,D,67.0,95.90,92.22,10.31,0.87,9.44,-2.792102
1,Aaron Ekblad,19-20,FLA,D,67,1220.55,56.75,50.83,69.36,52.87,...,20-21,FLA,D,35.0,132.75,91.82,21.66,1.93,19.73,7.497898
2,Aaron Ekblad,20-21,FLA,D,35,603.07,55.98,55.36,22.29,17.53,...,21-22,FLA,D,61.0,214.78,92.51,35.45,2.87,32.58,20.347898
3,Aaron Ekblad,21-22,FLA,D,61,1047.80,63.35,59.01,66.82,38.66,...,19-20,CHI,D,41.0,87.02,88.20,7.40,0.99,6.41,-5.822102
4,Aaron Ness,19-20,ARI,D,24,289.15,49.44,50.87,12.82,13.11,...,20-21,CHI,D,35.0,118.65,82.03,13.47,2.95,10.52,-1.712102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2650,Zdeno Chara,19-20,BOS,D,68,1166.42,62.20,50.51,57.18,34.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2651,Zdeno Chara,20-21,WSH,D,55,813.33,51.49,51.79,35.05,33.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2652,Zdeno Chara,21-22,NYI,D,72,1191.52,53.26,46.08,48.37,42.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2653,Zemgus Girgensons,19-20,BUF,C,69,783.73,45.67,49.54,22.91,27.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Drop duplicate rows
#combined_player_df = combined_player_df.drop_duplicates(subset=['Player','GP_x'])
#combined_player_df.head(20)

In [26]:
combined_player_df = combined_player_df.drop(columns = ['GF%_x','GF_x','GA_x','Season_y','Team_y','Position_y','GP_y','GF%_y','GF_y','GA_y','G+/-_y'])
combined_player_df

KeyError: ignored

In [ ]:
#Merge EV,PP,SH tables together
full_player_totals = pd.merge(combined_player_df, skater_SH_totals, how="left", on=["Player", "Player"])
#full_player_totals.head(10)

In [ ]:
#Clean up data
full_player_totals = full_player_totals.drop(columns = ['Season','Team','Position','GP','GF%','GF','GA','G+/-'])
#full_player_totals.head(10)

In [ ]:
#Merge EV,PP,SH and Points tables together
full_player_totals = pd.merge(full_player_totals, skater_box_totals, how="left", on=["Player", "Player"])
full_player_totals.head(30)

In [ ]:
combined_player_df = combined_player_df.drop_duplicates(subset=['Player','GP_x'])
combined_player_df.head(20)

In [ ]:
#Drop Repeated Columns
full_player_totals = full_player_totals.drop(columns = ['Season','Team','Position','GP','G','A1','A2'])
#full_player_totals.head(10)


In [ ]:
#Rename Columns
full_player_totals = full_player_totals.rename(columns = {'Season_x':'Season','Team_x':'Team','Position_x':'Position','GP_x':'GP','TOI_x':'TOI_EV','G+/-_x':'G_EV','xG+/-':'xG_EV','TOI_x':'TOI_PP','GAA_x':'GAA_PP','TOI_x':'TOI_SH','GAA_y':'GAA_SH','TOI_y':'TOI'})
#full_player_totals

In [ ]:
#Replace NaN with 0
full_player_totals = full_player_totals.replace(np.NaN, 0)
#full_player_totals

In [ ]:
#Import Goalie Data
goalie_stats_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Goalie%20Stats.csv')
#print(goalie_stats_raw)
#goalie_stats_raw.head()

In [ ]:
#Reduce Goalie Stats
goalie_stats = goalie_stats_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GA','Sv%','GSAx']]
#goalie_stats.head()

In [ ]:
#Add Active / Inactive Column to both data tables
full_player_totals['A/I'] = 'Active'
#full_player_totals

In [ ]:
#Make a Data Table for each team
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters

In [ ]:
full_player_totals.head(30)

In [ ]:
#Inspect Data
full_player_totals.head(20)

In [ ]:
#skater_PP_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/PP%20On-Ice%20Rates.csv')
#print(skater_PP_rates_raw)
#skater_PP_rates_raw.head()

**Stuff to Do Next - (have no more time, so quick thoughts that I have not checked, like the next one would be so easy to check if I had another 5 mins)**

In [ ]:
#Are these right? duplicate naming issues for "TOI_SH" and no PP?
full_player_totals.columns

Need to create a dataframe for each team. Use the loc function on player table first to get all the players for one team. Then do the same thing with goalies. We will probably need to keep them seperate tables since they do not have similar variables

Not necessary at this stage but eventually we are going to want to consolidate these code blocks. Ex, the code to create all of the team rosters should be in one block with no output. For now you can do 64 blocks (32 for skaters, 32 for goalies) and show output so we can see if there are any errors, but once we get past that stage we will comment out the output and put it all in one block. If we ever need to go back in check you just take out the '#' and run the code to see the output.

Need to add a column for 'status'. I am thinking the two states we use are 'active' and 'inactive' for the starting lineups. Will be 18 skaters and 1 goalie when we update daily but doesnt matter now. Should put this in before all of the loc functions so it shows up for the team dataframes. Note: will have to be added to both the full_player_totals dataframe and the goalie_stats dataframe.

Add folder in github for the csv's, update github links in code to pull data